In [93]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import scipy.stats as stats
import itertools
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [94]:
df = pd.read_csv('../Данные по использованию климатических систем.csv', sep=';')

In [95]:
params = {'text.color': "white", "font.size": 12}
plt.rcParams.update(params)
plt.tight_layout()
sns.set(style="whitegrid", font_scale=1.5)

<Figure size 640x480 with 0 Axes>

In [96]:
# PEP8

def make_word_pep8style(word: str) -> str:
    return word.lower().replace(' ', '_')

df = df.rename(make_word_pep8style, axis='columns')

list(df.columns)

['год',
 'время_года',
 'климат',
 'город',
 'страна',
 'способ_охлаждения',
 'режим_при_смешанном_типе_охлаждения',
 'способ_обогрева',
 'возраст',
 'пол',
 'ощущение_температуры',
 'ощущение_температуры_(bool)',
 'предпочтительное_изменение_температуры',
 'ощущение_движения_воздуха_(bool)',
 'предпочтительное_изменение_движения_воздуха',
 'оценка_комфорта',
 'утепление',
 'температура_воздуха_в_помещении',
 'температура_воздуха_на_улице',
 'rh',
 'скорость_воздуха',
 'рост',
 'вес',
 'занавески',
 'вентилятор',
 'окно',
 'двери',
 'отопление',
 'среднемесячная_температура_на_улице',
 'количество_рекламаций']

# Исследование данных на качество и предобработка

Разделим данные на числовые и категориальные

In [97]:
numerical = [
    'возраст',
    'температура_воздуха_в_помещении',
    'температура_воздуха_на_улице',
    'rh',
    'скорость_воздуха',
    'рост',
    'вес',
    'среднемесячная_температура_на_улице',
    'количество_рекламаций',
    'утепление',
    'ощущение_температуры',
    'оценка_комфорта',
]
categorial = list(set(df.columns) ^ set(numerical))

categorial_bool = [
    'ощущение_температуры_(bool)',
    'занавески',
    'вентилятор',
    'окно',
    'двери',
    'отопление'
]

## Первичный графический анализ

для поиска используя графический метод в категориальных факторах используем круговую диаграмму, а в количественных гистограммы и ящик с усами (boxplot)

### Для каждого числового столбца построим гистограмму и ящик с усами

In [ ]:
fig, ax = plt.subplots(len(numerical), 2)
fig.set_figheight(40)
fig.set_figwidth(13)
fig.tight_layout(h_pad=2)

for i, col in enumerate(numerical):
    sns.histplot(df[col], kde=True, ax=ax[i][0])
    sns.boxplot(df[col], ax=ax[i][1])
    ax[i][0].set_xlabel("")
    ax[i][0].set_title(col)

### А для каждого категориального столбца построим круговую диаграму

In [ ]:
fig, ax = plt.subplots((len(categorial) + 1) // 2, 2)
fig.set_figheight(30)
fig.set_figwidth(13)
fig.tight_layout(h_pad=0)
for i, col in enumerate(categorial):
    value_counts = pd.value_counts(df[col], dropna=False)
    ax[i // 2][i % 2].pie(value_counts, labels=value_counts.index)
    ax[i // 2][i % 2].set_title(col)

## Исправление орфографических ошибок (task2)

Обнаружены орфографические ошибки:
- В столбце **"предпочтительное_изменение_температуры"** *"Тепле"* и *"Холодн"*
- В столбце **"климат"** *"Субтроп океанич"*

In [ ]:
def fix_spelling_1st_problem(word: str) -> str:
    return "Теплее" if word == "Тепле" else "Холоднее" if word == "Холодн" else word
 
def fix_spelling_2nd_problem(word: str) -> str:
    return "Cубтропический океанический" if word == "Cубтроп океанич" else word

df["предпочтительное_изменение_температуры"] = df["предпочтительное_изменение_температуры"].apply(fix_spelling_1st_problem)
df["климат"] = df["климат"].apply(fix_spelling_2nd_problem)

# Проверка 

columns = ['предпочтительное_изменение_температуры', 'климат']

fig, ax = plt.subplots(1, 2)
fig.set_figheight(30)
fig.set_figwidth(13)
fig.tight_layout(h_pad=0)
for ind, col in enumerate(columns):
    value_counts = pd.value_counts(df[col], dropna=False)
    ax[ind].pie(value_counts, labels=value_counts.index)
    ax[ind].set_title(col)

## Исправление выбросов в росте (task5)

рост ~140 см считаем не аномалией

In [ ]:
def fix_height(num: int) -> int:
    return None if num < 135 else num

df['рост'] = df['рост'].apply(fix_height)

fig, ax = plt.subplots()
sns.boxplot(df['рост'], ax=ax)
ax.set_title("рост")

## Исправление выбросов в скорости воздуха (task5)

скорость > 5 м/с в помещении считаем аномальной

In [ ]:
# заменяем все показания в колонке скорость воздуха больше 30 на None, ибо такие показатели аномальные
df['скорость_воздуха'] = df['скорость_воздуха'].apply(lambda x: None if x > 5 or x is None else x)

sns.boxplot(df['скорость_воздуха'])

## Исследование колонок с температурой (task5)

Исследуем аномалии используя boxplot

In [ ]:
temperature_cols = [
    'температура_воздуха_в_помещении',
    'температура_воздуха_на_улице',
    'среднемесячная_температура_на_улице'
]
fig, ax = plt.subplots(3, 1)
fig.set_figwidth(11)
fig.set_figheight(15)
for i, col in enumerate(temperature_cols):
    sns.boxplot(data=df, x='страна', y=col, ax=ax[i])

Заметим, что у США наблюдаются выбросы в большую сторону. Предположим, что температура указана в °F. Тогда конвертируем все значения больше 40 из °F в °C. 

In [ ]:
def f2c(row):
    if row[17] is None:
        return None
    
    if row[4] == 'США':
        if row[17] > 40:
            return row[17] * 5 / 9 - 32
    return row[17]

df_new = pd.DataFrame()
df_new['страна'] = df['страна']
df_new['время_года'] = df['время_года']
df_new['температура_воздуха_в_помещении'] = df.apply(f2c, axis=1)
# print(df_new[df_new['время_года'] == 'Лето'])
fig, ax = plt.subplots(1, 1)
fig.set_figheight(8)
fig.set_figwidth(9)
sns.boxplot(data=df_new[df_new['время_года'] == 'Лето'], x='страна', y='температура_воздуха_в_помещении', ax=ax)

Поскольку после приведения температуры из °F в °C, оказалось, что в США летом температура в помещении составила в нижней квартили 12°C, поэтому гипотезу отвергаем. Значит мы имеем дело с выбросами

In [ ]:
def chnge_temperature(row, c):
    season = ['Лето', 'Зима', 'Осень', 'Весна']
    season_idx = season.index(row[1])
    col_idx = list(row.index).index(c)
    if lower_bound[season_idx] <= row[col_idx] <= upper_bound[season_idx]:
        return row[col_idx]
    return None



for col in temperature_cols:
    Q1 = [
        df[df['время_года'] == 'Лето'][col].quantile(0.25),
        df[df['время_года'] == 'Зима'][col].quantile(0.25),
        df[df['время_года'] == 'Осень'][col].quantile(0.25),
        df[df['время_года'] == 'Весна'][col].quantile(0.25),
    ]
    Q3 = [
        df[df['время_года'] == 'Лето'][col].quantile(0.75),
        df[df['время_года'] == 'Зима'][col].quantile(0.75),
        df[df['время_года'] == 'Осень'][col].quantile(0.75),
        df[df['время_года'] == 'Весна'][col].quantile(0.75),
    ]
    IQR = [
        Q3[0] - Q1[0],
        Q3[1] - Q1[1],
        Q3[2] - Q1[2],
        Q3[3] - Q1[3],
    ]
    lower_bound = [
        Q1[0] - 3 * IQR[0],
        Q1[1] - 3 * IQR[1],
        Q1[2] - 3 * IQR[2],
        Q1[3] - 3 * IQR[3]
    ]
    upper_bound = [
        Q3[0] + 3 * IQR[0],
        Q3[1] + 3 * IQR[1],
        Q3[2] + 3 * IQR[2],
        Q3[3] + 3 * IQR[3],
    ]
    df[col] = df.apply(chnge_temperature, axis=1, c=col)

fig, ax = plt.subplots(3, 1)
fig.set_figwidth(15)
fig.set_figheight(20)
for i, col in enumerate(temperature_cols):
    sns.boxplot(data=df, x='страна', y=col, ax=ax[i])

In [ ]:
df.isna().sum()

## Обработка дубликатов (удаляем) (task4)

In [ ]:
print(df.duplicated().value_counts())
df = df.drop_duplicates()

## Заполнение пропусков
Числовые пропуски заполняем сейчас, поскольку их довольно мало. Категориальные пропуски не трогаем до тех пор, пока не начнем с ними работать.

In [ ]:
'количество строк, которые имеют пропуски:', len(df[df['возраст'].isna()])

#### Возраст

In [ ]:
countries = df.groupby('страна')

In [ ]:
def fillna_age(row):
    if pd.isnull(row.loc['возраст']):
        return age_median[row.loc['страна']]
    return row.loc['возраст']


age_median = countries['возраст'].median()
df['возраст'] = df.apply(fillna_age, axis=1)

#### Скорость воздуха

In [ ]:
wind_velocity_median = df['скорость_воздуха'].median()
df['скорость_воздуха'] = df['скорость_воздуха'].apply(lambda x: wind_velocity_median if x is None else x)

#### Температура воздуха в помещении

In [ ]:
groups = df.groupby(['год', 'время_года', 'город'])['температура_воздуха_в_помещении'].median()

def fix_skips_temp_inside(row):
    if pd.isna(row['температура_воздуха_в_помещении']):
        return groups[row['год'], row['время_года'], row['город']]
    return row['температура_воздуха_в_помещении']

df['температура_воздуха_в_помещении'] = df.apply(fix_skips_temp_inside, axis=1)

#### Среднемесячная температура на улице

In [ ]:
temperature_groups = df.groupby(['город', 'год', 'время_года'])['среднемесячная_температура_на_улице'].median()
print(temperature_groups)

temperature_groups['Хайдарабад', 2012, 'Осень'] = (
    temperature_groups['Хайдарабад', 2012, 'Лето'] + temperature_groups['Хайдарабад', 2012, 'Зима']
) / 2

In [ ]:
def fillna_age(row):
    if pd.isnull(row['среднемесячная_температура_на_улице']):
        return temperature_groups[row['город'], row['год'], row['время_года']]
    return row['среднемесячная_температура_на_улице']


df['среднемесячная_температура_на_улице'] = df.apply(fillna_age, axis=1)

In [ ]:
# tools
def show_pie_charts(_col1, _col2):
    global df
    _columns = sorted(df[_col1].dropna().unique())
    _colors = ['green', 'blue', 'orange', 'pink', 'magenta', 'purple', 'red']
    _all_possible_values = df[_col2].unique()

    fig, ax = plt.subplots(len(_columns), 1)
    fig.set_figheight(15)
    fig.set_figwidth(10)
    fig.tight_layout(h_pad=0)
  
    for _ind, _col in enumerate(_columns):
        _vals = df[df[_col1] == _col][_col2].value_counts(dropna=True)
        wedges, _ = ax[_ind].pie(_vals, colors=_colors)
        ax[_ind].set_title(_col2)
        ax[_ind].legend(wedges, _all_possible_values,
          title=_col1 + ' = ' + str(_col),
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1))

# Проведение расчётов и исследований

## Создание категориального стоблца кол-ва рекламаций (task6)

In [ ]:
df['количество_рекламаций_кат'] = ['мало' if i <= 1 else ('средне' if i == 2 else 'много') for i in df['количество_рекламаций']]
df

## Создание категориального столбца по возрастным группам (task 9)

In [ ]:
df['возрастная_группа'] = ['молодой возраст' if i <= 44 else ('средний возраст' if 45 <= i <= 59 else 'пожилой возраст') for i in df['возраст']]

## Расчёт средней комфортной температуры в зависимости от возрастной категории (task7)

In [ ]:
df[df['ощущение_температуры_(bool)'] == 1].groupby(['возрастная_группа'])['температура_воздуха_в_помещении'].mean()

## Средний возраст респондентов по полу и стране (task 8)

По полу:

In [ ]:
df.groupby('пол')['возраст'].mean()


По стране:

In [ ]:
df.groupby('страна')['возраст'].mean()

In [ ]:
print(df.groupby(['страна', 'пол'])['возраст'].mean())

## Медианное значение влажности и температуры для каждого типа охлаждения (task 10)

In [ ]:
df.groupby('способ_охлаждения')[['температура_воздуха_в_помещении', 'rh']].median()

## Сводная таблица с данными, сгрупированными по стране, полу, возрастной группе(Task 11)

In [ ]:
print(df.groupby(['страна', 'пол', 'возрастная_группа'])[['температура_воздуха_в_помещении', 'температура_воздуха_на_улице', 'rh']].mean())

## Стандартная комфортная относительная влажность в помещениях (task 13)
- Согласно [СанПиН 2.2.4.548—96](https://kpfu.ru/portal/docs/F_979595184/SanPiN.2.2.4.548_96.o.mikroklimate.pdf) оптимальная влажность в рабочих помещениях в теплое и холодное время года должна составлять от $40\% - 60 \%$.
- Однако, согласно Управлению по охране труда США (Occupational Safety and Health Administration, OSHA) оптимальная влажность должна лежать в диапазоне от $20\% - 60\%$. [Ссылка на положение](https://www.osha.gov/laws-regs/standardinterpretations/2003-02-24).

Тем не менее, диапазон от $40$ до $60$ является более щадящим, что подтверждается приведенной ниже диаграммой. Поэтому в рамках нашего исследования мы будем опираться на него.
![](https://cndarcdn.scdn3.secure.raxcdn.com/m/0/diagramma-scofild-sterling-rus-01-1.jpg) [Страница с исследованиями в защиту диапазона от 40 до 60](https://40to60rh.com/)

In [ ]:
def rh2cat(rh):
    if 40 <= rh <= 60:
        return "Нормальная влажность"
    if rh < 40:
        return "Пониженная влажность"
    return "Повышенная влажность"

df['rh_кат'] = df['rh'].apply(rh2cat)

pd.value_counts(df['rh_кат']).plot.pie()
plt.show()

## Корреляция между параметрами (task14)

In [ ]:
# Рассматриваем cущественные корреляции Номинальная-Номаинальная 


for col1, col2 in itertools.combinations(list(df.columns), 2):
    if col1 in categorial and col2 in categorial:
        if col1 == col2:
            continue

        table = pd.crosstab(df[col1], df[col2])
        if len(table) == 0:
            continue
        
        res = stats.chi2_contingency(table)
        if res[1] < 0.05:
            n = table.sum().sum()
            phi2 = res[0] / n
            kramer = np.sqrt(phi2 / (min(table.shape) - 1))
            if kramer > 0.6:
                print(col1, col2, "  K=", kramer, "  p=", res[1])

- корреляции год - <что-то> являются особенностью данных (за 2010 собраны данные за один город, за 2011 за другой)
- климат - год, климат - страна: корреляции очевидны (города не перемещаются из одной климатической зоны в другую, по крайней мере в рамках наших данных)
- город - способ_охлаждения: эти данные нам также ничего не дают
- страна - <что-то>: особенности данных
- способ_охлаждение - отопление: это не имеет смысла, поскольку корреляция не видна графическим методом и данные содержат много пропусков. Если эти пропуски убрать, то pvalue значительно поднимется
- ощущение_движения_воздуха_(bool) - ощущение_температуры_(bool): видимо если людям нравится как двигается воздух, то им нравится температура
- ощущение_движения_воздуха_(bool) - предпочтительное_изменение_температуры: если людям нравится как движется воздух, то они не хотят менять температуру?
- занавески - вентилятор: наличие занавесок говорит нам о наличии вентрилятора у человека

In [ ]:
show_pie_charts('занавески', 'вентилятор')

In [ ]:
# Рассматриваем cущественные корреляции Количественная-Номаинальная 
temp_df = df.copy()
temp_df['способ_обогрева'] = temp_df['способ_обогрева'].fillna('нет')


for col1, col2 in itertools.product(list(categorial), list(numerical), repeat=1):
    vals = pd.get_dummies(df[col1], dummy_na=True)
    vals['ind'] = vals.apply(lambda row: list(row).index(1), axis=1)

    res = df[col2].corr(vals['ind'])

    if res > 0.5:
        print(col1, col2,  res)

In [ ]:
sns.boxplot(data=temp_df, x='ощущение_движения_воздуха_(bool)', y='утепление')

- ощущение_движения_воздуха_(bool) - утепление: если опрошенный считает движение воздуха хорошим, то он утепляется меньше. (средняя корреляция)

In [ ]:
sns.boxplot(data=temp_df, x='способ_обогрева', y='среднемесячная_температура_на_улице')

- способ_обогрева - среднемесячная_температура_на_улице: очевидно, что от среднемесичной теспературы будет зависеть, стоит ли у людей в доме отопление или нет. (средняя корреляция)

In [ ]:
sns.boxplot(data=temp_df, x='пол', y='рост')

- пол - рост
- климат - утепление: в жарких краях люди не вынуждены носить много теплой одежды
- страна - среднемесячная_температура_на_улице: погода (температура) циклична

In [ ]:
# Рассматриваем cущественные корреляции Количественная-Количественная

for col1, col2 in itertools.product(list(numerical), list(numerical), repeat=1):
    if col1 == col2:
        continue

    res = df[col1].corr(df[col2])

    if res > 0.5:
        print(col1, col2, res)

- температура_воздуха_в_помещении - температура_воздуха_на_улице: логично
- рост - вес: хорошего человека должно быть много

# Проверка гипотез

## Влияет ли способ охлаждения на оценку комфорта (task 15)

- Нулевая гипотеза: влияния/эффекта нет
- Альтернативная гипотеза: влияние/эффект есть

In [ ]:
temp_df = df[['способ_охлаждения', 'оценка_комфорта']].copy()
temp_df = temp_df.dropna()

sns.boxplot(x='способ_охлаждения', y='оценка_комфорта', data=temp_df)
plt.show()

# Шкалы разные: способ охлаждения - номинальная (3 уровня), оценка комфорта - количественная
# Проверка как распределена оценка комфорта

print("Распрление нормальное?: ", stats.shapiro(temp_df['оценка_комфорта'])[1] >= 0.05)

# Распределение не нормальное, поэтому применяем критерий Краскела-Уоллеса

g1 = temp_df[temp_df['способ_охлаждения'] == 'Смешанный']['оценка_комфорта']
g2 = temp_df[temp_df['способ_охлаждения'] == 'Вентиляция']['оценка_комфорта']
g3 = temp_df[temp_df['способ_охлаждения'] == 'Кондиционирование']['оценка_комфорта']
print("pvalue =", stats.kruskal(g1, g2, g3))

pvalue получилось более 0.05, значит мы не можем отвергнуть нулевую гипотезу о том, что фактор и отклик как-либо зависимы. То есть **подтвердить влияние типа охлаждения на оценку комфорта невозможно при наших данных.**

## Влияние пола на оценку комфорта (task16)

- Нулевая гипотеза: влияния/эффекта нет
- Альтернативная гипотеза: влияние/эффект есть

In [ ]:
temp_df = df[['пол', 'оценка_комфорта']].copy()
temp_df = temp_df.dropna()

sns.boxplot(x='пол', y='оценка_комфорта', data=temp_df)
plt.show()

# Шкалы разные: пол - номинальная (2 уровня), оценка комфорта - количественная
# Проверка как распределена оценка комфорта

print("Распрление нормальное?: ", stats.shapiro(temp_df['оценка_комфорта'])[1] >= 0.05)

# Распределение не нормальное, поэтому применяем метод Манна-Уитни

g1 = temp_df[temp_df['пол'] == 'Мужской']['оценка_комфорта']
g2 = temp_df[temp_df['пол'] == 'Женский']['оценка_комфорта']

print("pvalue =", stats.mannwhitneyu(g1, g2)[1])

# Если оценка_комфорта это номинальная шкала, то есть нужно применить хи-квадрат чтобы проверить корреляцию

table = pd.crosstab(df['пол'], df['оценка_комфорта'])

res = stats.chi2_contingency(table)

n = table.sum().sum()
phi2 = res[0] / n
kramer = np.sqrt(phi2 / (min(table.shape) - 1))
print("Если считать что кто оценка_комфорта это также номинальная шкала", "   K=", kramer, "  p=", res[1])

pvalue получилось более 0.05, значит мы не можем отвергнуть нулевую гипотезу о том, что фактор и отклик как-либо зависимы. То есть **подтвердить влияние пола на оценку комфорта невозможно при наших данных.**

## Влияет ли возрастная группа на оценку комфорта (task17)
- Нулевая гипотеза: влияния/эффекта нет
- Альтернативная гипотеза: влияние/эффект есть

In [ ]:
fig, ax = plt.subplots(figsize=(9,5))
sns.boxplot(df, x='возрастная_группа', y='оценка_комфорта', ax=ax)

Поскольку `возрастная_группа` представленна в виде категориальной шкалы, а `оценка_комфорта` в количественной, то будем использовать критерий Красскела-Уоллеса или ANOVA. Для это определим нормальность распределения в столбце `оценка_комфорта` 

In [ ]:
sns.histplot(df, x='оценка_комфорта', kde=True, bins=5)

if stats.shapiro(df['оценка_комфорта'])[1] >= 0.5:
    print('Распределение \033[1mнормальнное\033[0m, исппользуем ANOVA')
else:
    print('Распределение \033[1mненормальное\033[0m, используем критерий Красскела-Уоллеса')

In [ ]:
gp = df.groupby('возрастная_группа')['оценка_комфорта']
anova = stats.f_oneway(*(v for v in gp.groups.values()))
anova

In [ ]:
if anova[1] >= 0.05:
    print('Поскольку p-значение не меньше 0,05, мы \033[1mне\033[0m можем отвергнуть нулевую гипотезу.\n'
          'Следовательно, у нас нет достаточных доказательств того, что возрастная группа влияет на оценку комфорта.')
else:
    print('Поскольку p-значение меньше 0,05, мы можем отвергнуть нулевую гипотезу.\n'
          'Следовательно, у нас достаточно доказательств того, что возрастная группа \033[1mвлияет\033[0m на оценку комфорта.')

## Взаимосвязь между количеством рекламаций и оценкой комфорта (task18)
- Нулевая гипотеза: влияния/эффекта нет
- Альтернативная гипотеза: влияние/эффект есть

In [ ]:
temp_df = df[['оценка_комфорта', 'количество_рекламаций']].copy()
temp_df = temp_df.dropna()

print("корелляция и pvalue: ", stats.pearsonr(temp_df['количество_рекламаций'], temp_df['оценка_комфорта']))

sns.regplot(df, x='количество_рекламаций', y='оценка_комфорта')
plt.show()

Корреляция (0.5) не сильная, однако и не нулевая. Pvalue намного меньше 0.05, поэтому мы можем доверять полученной корреляции. *То есть взаимосвязь есть, однако она не сильная.*

## Проверить гипотезу о том, что средняя оценка комфорта отличается в зависимости от страны (task19)
- Нулевая гипотеза: влияния/эффекта нет
- Альтернативная гипотеза: влияние/эффект есть

In [ ]:
pd.read_csv('../Данные по использованию климатических систем.csv', sep=';').groupby('Страна')['Ощущение движения воздуха (bool)'].sum()

In [ ]:
fig, ax = plt.subplots(figsize=(9,5))
sns.boxplot(df, x='страна', y='оценка_комфорта', ax=ax)

По графику не трудно заметить, что средняя оценка у Индии больше чем у США, а у Австралии вообще нет оценок.
Впрочем это можно подтвердить математически

In [ ]:
df.groupby('страна')['оценка_комфорта'].mean()

Итак, средняя оценка комфортра отличается в зависимости от страны

## Выдвинуть и проверить гипотезу по влиянию разных факторов на оценку комфорта/ощущение температуры/ощущение движения воздуха (task 21)

### Оценка комфорта

**Гипотеза**
> **Категории относительная влажность** влияет на **оценку комфорта**  

- Нулевая гипотеза: влияния/эффекта нет
- Альтернативная гипотеза: влияние/эффект есть

In [ ]:
# Поскольку колонки 

## Влияние разных факторов на оценку комфорта/ощущение температуры/ощущение движения воздуха (task 22)

### Оценка комфорта

**Гипотеза**
> **Категории относительная влажность** влияет на **оценку комфорта**  

- Нулевая гипотеза: влияния/эффекта нет
- Альтернативная гипотеза: влияние/эффект есть

In [ ]:
# Поскольку колонки rh_кат и оценка_комфорта представленны разными типами
# А в rh_кат более двух уровней, будем использовать ANOVA или критерий Красскела-Уоллеса

temp_df = df[['rh_кат', 'оценка_комфорта']].copy().dropna()

gp = temp_df.groupby('rh_кат')
if stats.shapiro(temp_df.оценка_комфорта)[1] >= 0.5:
    temp = stats.f_oneway(*(v for v in gp.groups.values()))
else:
    temp = stats.kruskal(*(v for v in gp.groups.values()))

print(temp)

if temp[1] >= 0.05:
    print('Поскольку p-значение не меньше 0,05, мы \033[1mне\033[0m можем отвергнуть нулевую гипотезу.\n'
          'Следовательно, у нас нет достаточных доказательств того, что категории относительной влажности влияют на оценку комфорта.')
else:
    print('Поскольку p-значение меньше 0,05, мы можем отвергнуть нулевую гипотезу.\n'
          'Следовательно, у нас достаточно доказательств того, что категории относительной влажности \033[1mвлияет\033[0m на оценку комфорта.')

### Ощущение температуры

**Гипотеза**
> принадлежность респондента к возрастной группе влияет на его ощущение температуры  

- Нулевая гипотеза: влияния/эффекта нет
- Альтернативная гипотеза: влияние/эффект есть

In [ ]:
# Поскольку колонки возрастная_группа и ощущение_температуры представленны разными типами
# А в возрастная_группе более двух уровней, будем использовать ANOVA или критерий Красскела-Уоллеса

temp_df = df[['возрастная_группа', 'ощущение_температуры']].copy().dropna()

gp = temp_df.groupby("возрастная_группа")
if stats.shapiro(temp_df.ощущение_температуры)[1] >= 0.5:
    temp = stats.f_oneway(*(v for v in gp.groups.values()))
else:
    temp = stats.kruskal(*(v for v in gp.groups.values()))

print(temp)

if temp[1] >= 0.05:
    print('Поскольку p-значение не меньше 0,05, мы \033[1mне\033[0m можем отвергнуть нулевую гипотезу.\n'
          'Следовательно, у нас нет достаточных доказательств того, что принадлежность к возрастной группе влияет на оценку комфорта')
else:
    print('Поскольку p-значение меньше 0,05, мы можем отвергнуть нулевую гипотезу.\n'
          'Следовательно, у нас достаточно доказательств того, что принадлежность к возрастной группе влияет на оценку комфорта.')

### Ощущение движения воздуха

**Гипотеза**
> В работе ...

- Нулевая гипотеза: влияния/эффекта нет
- Альтернативная гипотеза: влияние/эффект есть

In [ ]:
temp_df = df[['ощущение_движения_воздуха_(bool)', 'rh_кат']].copy().dropna()

crosstab = pd.crosstab(temp_df['ощущение_движения_воздуха_(bool)'], temp_df['rh_кат'])
chi2 = stats.chi2_contingency(crosstab)[0]
n = crosstab.sum().sum()
phi2 = chi2 / n
kramer = np.sqrt(phi2 / (min(crosstab.shape) - 1))

print(kramer)

## Какая скорость воздуха будет комфортной, и есть ли корреляция оценки воздуха с его реальной скоростью(task20)

In [ ]:
temp_df = df[['ощущение_движения_воздуха_(bool)', 'скорость_воздуха']].copy().dropna()

plt.figure(figsize=(10,10))
sns.boxplot(data=temp_df, x='ощущение_движения_воздуха_(bool)', y='скорость_воздуха')

In [ ]:
_, p = stats.shapiro(temp_df['скорость_воздуха'])

if p > 0.05:
    print('распределение нормальное')
else:
    print('распределение ненормальное')

# Распределение ненормальное поэтому используем метод Манна-Уитни, чтобы исследовать корреляцию

g1 = temp_df[temp_df['ощущение_движения_воздуха_(bool)'] == 1]['скорость_воздуха']
g2 = temp_df[temp_df['ощущение_движения_воздуха_(bool)'] == 0]['скорость_воздуха']

_, p = stats.mannwhitneyu(g1, g2, alternative='two-sided')
print('pvalue = ', p)

pvalue меньше 0.05, поэтому мы отвергаем нулевую гипотезу и принимаем альтернативную, что ***распределния не равны и имеют статистически значимые отличия*** </br> Также по графику обратим внимание на медианы и распределение в целом: ***чем меньше скорость воздуха, тем больше шанс, что респондента устраивает скорость воздуха в помещении***

In [ ]:
temp_df = df[['оценка_комфорта', 'скорость_воздуха']].copy().dropna()

sns.regplot(data=temp_df, x='оценка_комфорта', y='скорость_воздуха')

In [ ]:
_, p = stats.shapiro(temp_df['скорость_воздуха'])

if p > 0.05:
    print('распределение нормальное')
else:
    print('распределение ненормальное')

# Распределение ненормальное поэтому используем корреляцию Спирмена

_, p = stats.spearmanr(temp_df['скорость_воздуха'], temp_df['оценка_комфорта'])
print('pvalue = ', p)
temp_df['скорость_воздуха'].corr(temp_df['оценка_комфорта'], method='spearman')

# Регрессионное моделирование


# Выводы

In [ ]:
df['климат_способ'] = df['климат'] + '_' + df['способ_охлаждения']

In [ ]:
temp_df = df[['климат_способ', 'оценка_комфорта']].copy().dropna()
# --------------------------------------------------------------------------------------------------------------------------

if stats.shapiro(temp_df.оценка_комфорта)[1] >= 0.5:
    temp = stats.f_oneway(*(v for v in gp.groups.values()))
else:
    temp = stats.kruskal(*(v for v in gp.groups.values()))

print(temp)

In [ ]:
sns.boxplot(temp_df, x='оценка_комфорта', y='климат_способ', order=sorted(temp_df['климат_способ'].unique(), key=lambda x: temp_df[temp_df['климат_способ'] == x]['оценка_комфорта'].median()))